# Corner Detection Experiment

This notebook focuses on **Corner Detection** methods to identify ROIs.
QR codes are rich in corners (finder patterns, alignment patterns, data modules).

We implement two standard filters:
1.  **Harris Corner Detection**: Measures "cornerness" based on eigenvalues differences.
2.  **Shi-Tomasi (MinEigenVal)**: Measures the minimum eigenvalue directly (often more stable).

**Note on Dilation**: Since corners are typically sparse points, we apply a **dilation** step to connect nearby corners into a dense blob. This allows the simple sliding-window ROI extractor to find a high-score "region" rather than just a single pixel.

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from pathlib import Path

%matplotlib inline

## 1. Filter Implementations

In [2]:
def normalize01(x: np.ndarray) -> np.ndarray:
    if x is None: return None
    x = x.astype(np.float32)
    mn, mx = float(x.min()), float(x.max())
    if mx - mn < 1e-6:
        return np.zeros_like(x, dtype=np.float32)
    return (x - mn) / (mx - mn)

# --- 1. Harris Corner --- 
def filter_harris(gray, blockSize=2, ksize=3, k=0.04, dilate_iter=1):
    # Standard Harris
    dst = cv2.cornerHarris(gray, blockSize, ksize, k)
    
    # Thresholding? Harris output is raw score. We normalize it first.
    norm = normalize01(dst)
    
    # Dilation to connect corners into a blob
    if dilate_iter > 0:
        kernel = np.ones((3,3), np.uint8)
        norm = cv2.dilate(norm, kernel, iterations=dilate_iter)
        
    return norm

# --- 2. Shi-Tomasi (Min EigenVal) ---
def filter_shi_tomasi(gray, blockSize=2, ksize=3, dilate_iter=1):
    # Min EigenVal
    dst = cv2.cornerMinEigenVal(gray, blockSize, ksize)
    
    norm = normalize01(dst)
    
    if dilate_iter > 0:
        kernel = np.ones((3,3), np.uint8)
        norm = cv2.dilate(norm, kernel, iterations=dilate_iter)
    
    return norm

## 2. Interactive Experiment

In [3]:
# Load Images
data_dir = Path("../data/raw")
files = sorted(list(data_dir.glob("*.png")) + list(data_dir.glob("*.jpg")) + list(data_dir.glob("*.jpeg")))
options = [(p.name, str(p)) for p in files]
if not options: options = [("No images", None)]

def extract_boxes(img_path, score_map, patch_size=128, top_k=50):
    img = cv2.imread(img_path)
    if img is None: return None
    
    h, w = score_map.shape[:2]
    s_map = score_map.copy()
    half = patch_size // 2
    out_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # 1. Collect Candidates
    candidates = [] 
    scores = []
    
    search_limit = 1000
    for _ in range(search_limit):
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(s_map)
        if max_val < 0.05: break
        
        cx, cy = max_loc
        x1 = max(0, cx - half); y1 = max(0, cy - half)
        x1 = int(x1); y1 = int(y1)
        
        candidates.append([x1, y1, patch_size, patch_size])
        scores.append(float(max_val))
        
        # Minimal suppression to find next peak
        pad = int(patch_size * 0.2)
        nx1 = max(0, cx - pad//2); ny1 = max(0, cy - pad//2)
        nx2 = min(w, cx + pad//2); ny2 = min(h, cy + pad//2)
        s_map[ny1:ny2, nx1:nx2] = 0

    # 2. NMS (IoU 0.6)
    if not candidates: return out_img, 0
    
    indices = cv2.dnn.NMSBoxes(candidates, scores, score_threshold=0.05, nms_threshold=0.6)
    count = 0
    if len(indices) > 0:
        for i in indices.flatten():
            if count >= top_k: break
            x, y, wb, hb = candidates[i]
            x2 = min(w, x + wb); y2 = min(h, y + hb)
            cv2.rectangle(out_img, (x, y), (x2, y2), (0, 255, 0), 2)
            count += 1
            
    return out_img, count


def show_corners(
    image_path,
    # Harris
    h_block, h_k, h_dilate,
    # Shi-Tomasi
    st_block, st_dilate,
    # Extraction
    patch_size
):
    if not image_path: return
    
    img = cv2.imread(image_path)
    if img is None: return
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # --- Compute Maps ---
    map_harris = filter_harris(gray, blockSize=h_block, k=h_k, dilate_iter=h_dilate)
    map_shi = filter_shi_tomasi(gray, blockSize=st_block, dilate_iter=st_dilate)
    
    # --- Extract Boxes ---
    res_harris, count_h = extract_boxes(image_path, map_harris, patch_size=patch_size, top_k=50)
    res_shi, count_s = extract_boxes(image_path, map_shi, patch_size=patch_size, top_k=50)
    
    # --- Visualize ---
    fig, axes = plt.subplots(2, 2, figsize=(14, 14))
    
    # Row 1: Harris
    im1 = axes[0, 0].imshow(map_harris, cmap='jet')
    axes[0, 0].set_title("Harris Score Map")
    axes[0, 0].axis('off')
    plt.colorbar(im1, ax=axes[0, 0], fraction=0.046, pad=0.04)
    
    axes[0, 1].imshow(res_harris)
    axes[0, 1].set_title(f"Harris Result ({count_h} Boxes)")
    axes[0, 1].axis('off')
    
    # Row 2: Shi-Tomasi
    im2 = axes[1, 0].imshow(map_shi, cmap='jet')
    axes[1, 0].set_title("Shi-Tomasi Score Map")
    axes[1, 0].axis('off')
    plt.colorbar(im2, ax=axes[1, 0], fraction=0.046, pad=0.04)
    
    axes[1, 1].imshow(res_shi)
    axes[1, 1].set_title(f"Shi-Tomasi Result ({count_s} Boxes)")
    axes[1, 1].axis('off')
    
    plt.tight_layout()
    plt.show()

interact(
    show_corners, 
    image_path=widgets.Dropdown(options=options, description="Image:"),
    
    # Harris
    h_block=widgets.IntSlider(value=2, min=2, max=7, description='Harris Block'),
    h_k=widgets.FloatSlider(value=0.04, min=0.01, max=0.2, step=0.01, description='Harris k'),
    h_dilate=widgets.IntSlider(value=2, min=0, max=10, description='Harris Dilate'),
    
    # Shi-Tomasi
    st_block=widgets.IntSlider(value=2, min=2, max=7, description='Shi Block'),
    st_dilate=widgets.IntSlider(value=2, min=0, max=10, description='Shi Dilate'),
    
    patch_size=widgets.IntSlider(value=128, min=32, max=256, step=32, description='Patch Size'),
);

interactive(children=(Dropdown(description='Image:', options=(('001.png', '../data/raw/001.png'), ('002.png', …